# 🧠 NeuroATMv2: Real Third Path Emergence
This version tracks true third-path behavior (action = 3), with selective memory and stable context.

In [ ]:

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Redesigned TinyNN with an extra hidden layer for more reliable context encoding
class TinyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 2)
        )

    def forward(self, x):
        return self.net(x)

# Memory-driven agent with selective conflict resolution and real third path
class NeuroATMv2:
    def __init__(self, memory_limit=100, conflict_threshold=0.01):
        self.memory = []
        self.memory_limit = memory_limit
        self.threshold = conflict_threshold

    def act(self, context):
        if not self.memory:
            return np.random.randint(2)

        sims = [np.dot(context, mem[0]) for mem in self.memory]
        if len(sims) < 2:
            return self.memory[np.argmax(sims)][1]

        top = np.argsort(sims)[-2:]
        if abs(sims[top[0]] - sims[top[1]]) < self.threshold:
            # Conflict detected, spawn new third-path behavior
            new_response = 3
            self._add_to_memory(context, new_response, initial_score=1)
            return new_response
        return self.memory[top[0]][1]

    def update(self, context, response, success):
        for i, (ctx, resp, score) in enumerate(self.memory):
            if np.allclose(ctx, context, atol=0.1) and resp == response:
                self.memory[i] = (ctx, resp, score + success)
                return
        self._add_to_memory(context, response, initial_score=success)

    def _add_to_memory(self, context, response, initial_score=0):
        if len(self.memory) >= self.memory_limit:
            self.memory.pop(0)
        self.memory.append((context, response, initial_score))

# Generate data near the decision boundary
def generate_conflict_data(n=100):
    np.random.seed(42)
    X = np.random.uniform(-0.1, 0.1, (n, 2))
    Y = np.random.randint(0, 2, size=n)
    return X, Y

# Setup
X, Y = generate_conflict_data()
model = TinyNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# Pretrain for more stability
for _ in range(300):
    inputs = torch.tensor(X, dtype=torch.float32)
    labels = torch.tensor(Y, dtype=torch.long)
    output = model(inputs)
    loss = loss_fn(output, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Agent simulation
agent = NeuroATMv2()
log = []
conflict_log = []
memory_sizes = []

for i in range(len(X)):
    with torch.no_grad():
        ctx = model(torch.tensor(X[i], dtype=torch.float32)).numpy()
    action = agent.act(ctx)
    if action == 3:
        conflict_log.append(i)
    reward = 1 if action == Y[i] else -1 if action in [0, 1] else 0
    agent.update(ctx, action, reward)
    log.append(action)
    memory_sizes.append(len(agent.memory))

# Plotting
plt.figure(figsize=(10, 4))
plt.plot(log, label='Action: 0/1, 3=Third Path', lw=1.5)
plt.scatter(conflict_log, [log[i] for i in conflict_log], color='red', label='Third Path Triggered', zorder=5)
plt.title('Agent Actions with Explicit Third Path')
plt.xlabel('Iteration')
plt.ylabel('Action')
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(memory_sizes, label='Memory size over time', color='green')
plt.title('Selective Memory Expansion')
plt.xlabel('Iteration')
plt.ylabel('Memory Size')
plt.grid(True)
plt.legend()
plt.show()
